### Importing packages

In [1]:
import torch
import torch.nn as nn
from torch.optim import SGD,Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Fetching essential features

In [2]:
df=pd.read_csv(r"C:\Users\Micro\Downloads\archive (3)\Bank_Personal_Loan_Modelling.csv")
df.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [3]:
col=df.columns
col

Index(['ID', 'Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [4]:
col=['Age','Experience','Income','Family','CCAvg','Education','Mortgage','Securities Account','CD Account',\
 'Online','CreditCard','Personal Loan']
df=df[col]

### Checking for class imbalance

In [5]:
df["Personal Loan"].value_counts()

0    4520
1     480
Name: Personal Loan, dtype: int64

### Standardization

In [6]:
x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [7]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)

### Resolving class imbalance

In [8]:
from imblearn.over_sampling import SMOTE
smote=SMOTE()
x,y=smote.fit_resample(x,y)

In [9]:
x.shape

(9040, 11)

In [10]:
y=y.reshape((9040,1))
y.shape

(9040, 1)

### Constructing NN

In [11]:
x=torch.tensor(x,dtype=torch.float32)
y=torch.tensor(y,dtype=torch.float32)

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

In [13]:
model= nn.Sequential(nn.Linear(11,5) ,nn.Sigmoid(), nn.Linear(5,1), nn.Sigmoid())

In [14]:
loss_fn=nn.BCELoss()

In [15]:
for i in model.state_dict():
    print(i)

0.weight
0.bias
2.weight
2.bias


### Particle swarm optimization

In [241]:
iterations=50
particles_n=10
particles=[]
for i in range(particles_n):
    temp=np.random.normal(0,1,66)
    particles.append(temp)
particles=np.array(particles)
velocity=np.random.normal(0,1,(particles_n))
it_particles=[]

In [242]:
p=np.full((iterations,particles_n),100).astype("float")
p_best_ind=np.full(particles_n,100).astype("int")

In [243]:
def obj_fn(weights):
    sd={}
    sd["0.weight"]=torch.tensor(weights[:55].reshape(5,11))
    sd["0.bias"]=torch.tensor(weights[55:60])
    sd["2.weight"]=torch.tensor(weights[60:65].reshape(1,5))
    sd["2.bias"]=torch.tensor(weights[65:])
    model.load_state_dict(sd)
    pred=model(x)
    loss=loss_fn(pred,y)
    return loss.item()

In [244]:
w=0.9
c1=0.2
c2=0.4

In [245]:
for i in range(iterations):
    r1=np.random.randn()/10
    r2=np.random.randn()/10
    it_particles.append(particles)
    for j in range(particles_n):
        p[i,j]=obj_fn(particles[j])
        p_best_ind[j]=np.argmin(p[:,j])
        p_best=it_particles[p_best_ind[j]][j]
        particles[j]+=w*velocity[j]+c1*r1*np.absolute(p_best-particles[j])
    t1=np.amin(p_best_ind) #best iteration
    t2=np.argmin(p_best_ind) #best particle
    g_best=it_particles[t1][t2]
    for j in range(particles_n):
        particles[j]+=c2*r2*np.absolute(g_best-particles[j])
    it_particles.append(particles)
    w=w-0.02